# Introduction to GEE with python

conda create -n gee python=3.8

conda activate gee

pip install earthengine-api geemap windows-curses

conda install jupyter

conda install ipykernel

python -m ipykernel install --user --name gee --display-name "Python (gee)"

jupyter notebook

# GEE WITH PYTHON

In [5]:
import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize()

# Section 1

In [19]:
# Section 1 : Données Vectorielles
# Créer une carte interactive pour les données vectorielles
Map_vector = geemap.Map()

# Importer les pays (0-3 levels shapefiles)
pays = ee.FeatureCollection('projects/ee-papaamadouniang2004/assets/sen_admbnda_adm0_anat_20240520')
regions = ee.FeatureCollection('projects/ee-papaamadouniang2004/assets/sen_admbnda_adm1_anat_20240520')
departements = ee.FeatureCollection('projects/ee-papaamadouniang2004/assets/sen_admbnda_adm2_anat_20240520')
communes = ee.FeatureCollection('projects/ee-papaamadouniang2004/assets/sen_admbnda_adm3_anat_20240520')


# Calculs statistiques : nombre de géométries, CRS, extent, projection
country_count = pays.size().getInfo()
print(f'Nombre de géométries (pays) : {country_count}')

crs_country = pays.first().geometry().projection().getInfo()
print(f'CRS du shapefile du pays : {crs_country}')

extent_country = pays.geometry().bounds().getInfo()
print(f'Extent (limites) du pays : {extent_country}')

# Calculs statistiques 2 : centroïdes, aire, périmètre
centroids = pays.map(lambda feature: feature.setGeometry(feature.geometry().centroid()))
country_area = pays.geometry().area().getInfo()
print(f'Aire totale du pays (en mètres carrés) : {country_area}')

country_perimeter = pays.geometry().perimeter().getInfo()
print(f'Périmètre total du pays (en mètres) : {country_perimeter}')

# Visualisation
Map_vector.addLayer(pays, {}, 'Pays')
Map_vector.addLayer(centroids, {'color': 'red'}, 'Centroïdes des pays')
Map_vector.centerObject(pays, 3)

# Afficher la carte
Map_vector

Nombre de géométries (pays) : 1
CRS du shapefile du pays : {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Extent (limites) du pays : {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-17.53114756482535, 12.308126642857681], [-11.345215008418714, 12.308126642857681], [-11.345215008418714, 16.69280533655572], [-17.53114756482535, 16.69280533655572], [-17.53114756482535, 12.308126642857681]]]}
Aire totale du pays (en mètres carrés) : 197514802862.18713
Périmètre total du pays (en mètres) : 3441351.1148358406


Map(center=[14.358557193312047, -14.466239286232163], controls=(WidgetControl(options=['position', 'transparen…

# Section 2

In [18]:
# Section 2
# Fonction pour obtenir les images raster par année et sélectionner la première bande
def get_raster_by_year(year):
    raster_name = f"202406_Global_Pf_Parasite_Rate_SEN_{year}"
    asset_id = f'projects/ee-papaamadouniang2004/assets/{raster_name}'
    return ee.Image(asset_id).select(0)

# Liste des années pour les rasters
years = list(range(2000, 2023))

# Charger toutes les images raster et sélectionner la première bande
rasters = [get_raster_by_year(year) for year in years]

# Créer une collection à partir de la liste d'images
raster_collection = ee.ImageCollection(rasters)

# Calculer les statistiques sur la collection
mean_image = raster_collection.mean()
median_image = raster_collection.median()
std_image = raster_collection.reduce(ee.Reducer.stdDev())
min_image = raster_collection.min()
max_image = raster_collection.max()

# Paramètres de visualisation
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Créer une instance de carte unique
Map = geemap.Map(center=[0, 0], zoom=2)

# Fonction pour visualiser une image en ajoutant une couche à la carte
def visualize_image(image, vis_params, title):
    Map.addLayer(image, title, vis_params)

# Visualiser chaque statistique
visualize_image(mean_image, 'mean_image' , vis_params)

visualize_image(median_image, 'median_image', vis_params)

visualize_image(std_image,'std_image', vis_params)

visualize_image(min_image,'min_image', vis_params)

visualize_image(max_image,'max_image', vis_params)

# Afficher la carte avec toutes les couches ajoutées
Map.add_colorbar(vis_params, label='Indice du Paludisme (2000-2022)')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…